In [1]:
# !pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

In [2]:
import os
import json
from datetime import datetime, timezone, timedelta
import time
import uuid
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import WMLCredentialsCP4D
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Create Dummy Headless ML Provider in OpenScale
This is used when there is no need for OpenScale to communicate with the deployment (e.g., send scoring request to the endpoint).

In [3]:
from ibm_cloud_sdk_core.authenticators import BearerTokenAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest

In [4]:
authenticator = BearerTokenAuthenticator(
        bearer_token=os.environ['USER_ACCESS_TOKEN']
    )

wos_client = APIClient(authenticator=authenticator, 
                       service_url=os.environ['RUNTIME_ENV_APSX_URL'])
wos_client.version

'3.0.7'

In [5]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2022-01-13 06:19:21.327000+00:00,00000000-0000-0000-0000-000000000000


In [6]:
data_marts = wos_client.data_marts.list().result.data_marts
DATA_MART_ID = data_marts[0].metadata.id
DATA_MART_ID 
print("Data Mart ID: {}".format(DATA_MART_ID))

Data Mart ID: 00000000-0000-0000-0000-000000000000


## Create headless service provider

In [7]:
SERVICE_PROVIDER_NAME = "OpenScale Headless Service Provider"
SERVICE_PROVIDER_DESCRIPTION = ""

In [8]:
existing_providers_dict = wos_client.service_providers.list().get_result().to_dict()['service_providers']
existing_providers = [sp['entity']['name'] for sp in existing_providers_dict]

if not SERVICE_PROVIDER_NAME in existing_providers:
    added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        operational_space_id = "production",
        credentials=WMLCredentialsCP4D(),
        background_mode=False
     ).result
    
    SERVICE_PROVIDER_ID = added_service_provider_result.metadata.id
    print(f"Service provider ID: {SERVICE_PROVIDER_ID}")
else:
    SERVICE_PROVIDER_ID = next((sp['metadata']['id'] for sp in existing_providers_dict \
                                if sp['entity']['name'] == SERVICE_PROVIDER_NAME))
    print(f"Service provider ID: {SERVICE_PROVIDER_ID}")

Service provider ID: 1059fa1d-7dfc-4d7b-8e40-ae8aef72afe1


In [9]:
wos_client.service_providers.show()

None,active,OpenScale Headless Service Provider - Test,custom_machine_learning,2022-02-23 18:40:27.254000+00:00,046a4364-6c34-4a66-ae25-ed83a9f764ec
None,active,OpenScale Headless Service Provider,custom_machine_learning,2022-02-07 19:41:21.084000+00:00,1059fa1d-7dfc-4d7b-8e40-ae8aef72afe1
99999999-9999-9999-9999-999999999999,active,DeepLIIF WML Prod,watson_machine_learning,2022-01-13 19:11:00.897000+00:00,09d6e718-c51f-41e2-9655-3991b6ac0104
99999999-9999-9999-9999-999999999999,active,DeepLIIF WML Pre-Prod,watson_machine_learning,2022-01-13 19:10:38.638000+00:00,1f4fab35-d955-4b91-a1f3-3dab72491116
99999999-9999-9999-9999-999999999999,active,WOS ExpressPath WML pre_production binding,watson_machine_learning,2022-01-13 06:19:31.996000+00:00,ae0d9dae-8599-400a-bcb9-0b5ab9e004f6
